In [1]:
import sys
sys.path.append("../..")
import numpy as np
from iirc.datasets_loader import get_lifelong_datasets
from iirc.definitions import PYTORCH, IIRC_SETUP
from iirc.utils.download_cifar import download_extract_cifar100
from __future__ import print_function
from __future__ import division

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

from torchvision.datasets import CIFAR100
import torchvision.transforms as tt
import torch.nn.functional as F

import time
import os
import copy
print("PyTorch Version: ", torch.__version__)
print("Torchvision Version: ", torchvision.__version__)

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

PyTorch Version:  1.13.0
Torchvision Version:  0.14.0
device: cuda:0


In [2]:
download_extract_cifar100("../../data")

downloading CIFAR 100
dataset downloaded
extracting CIFAR 100
dataset extracted


In [3]:
import torchvision.transforms as transforms

essential_transforms_fn = transforms.ToTensor()
augmentation_transforms_fn = transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

In [4]:
dataset_splits, tasks, class_names_to_idx = \
    get_lifelong_datasets(dataset_name = "iirc_cifar100",
                          dataset_root = "../../data", # the imagenet folder (where the train and val folders reside, or the parent directory of cifar-100-python folder
                          setup = IIRC_SETUP,
                          framework = PYTORCH,
                          tasks_configuration_id = 0,
                          essential_transforms_fn = essential_transforms_fn,
                          augmentation_transforms_fn = augmentation_transforms_fn,
                          joint = False
                         )

Creating iirc_cifar100
Setup used: IIRC
Using PyTorch
Dataset created


In [5]:
# print(len(tasks))
n_classes_per_task = []
for task in tasks:
    n_classes_per_task.append(len(task))
n_classes_per_task = np.array(n_classes_per_task)

In [27]:
# lifelong_datasets['train'].choose_task(2)
# print(list(zip(*lifelong_datasets['train']))[1])
for i in dataset_splits:
    print(i)
dataset_splits["test"].choose_task(1)
print(list(zip(*dataset_splits["test"]))[1])

train
intask_valid
posttask_valid
test
('small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_mammals', 'small_m

In [7]:
# initialize a pretrained model (imageNet)
model_name = "resnet" #choosing alexnet since it is "relatively" easy to train
# model_name = "squeezenet" # changed to squeezeNet since it gets same acc as alex but smaller
num_classes = 9 # in cifar100

batch_size = 8

num_epochs = 15

feature_extract = False #set to false so we can finetune entire model

In [8]:
def train_model(model, trainloader, criterion, optimizer, num_classes, num_epochs=5 ):
    since = time.time() # including this just because
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        
                
        running_loss = 0.0
        running_corrects = 0

        # iterate over data
        for inputs,label1,label2 in trainloader:
            inputs = inputs.to(device)
            label1 = torch.from_numpy(np.array([class_names_to_idx[i] for i in label1]))
            label1 = F.one_hot(label1, num_classes=num_classes)
            label1 = label1.to(torch.float32)
            label1 = label1.to(device)
#             label2 = label2.to(device)


            #empty the gradients
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, label1)
            loss.backward()
            optimizer.step()





            # statistics
            running_loss += loss.item() * inputs.size(0)
#             running_corrects += torch.sum(preds == labels.data)
                
        epoch_loss = running_loss / len(trainloader.dataset)
        print("len dataset = ",len(trainloader.dataset))
#             epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
        print('{} Loss: {:.4f}'.format('train', epoch_loss))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    # load best model weights
    return model

In [9]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [10]:
class MultilabelClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
        
        self.model_wo_fc = nn.Sequential(*(list(self.resnet.children())[:-1]))
        self.num_ftrs = self.resnet.fc.in_features
        
        self.fc = nn.Linear(self.num_ftrs, num_classes)
        
    def forward(self, x):
        x = self.model_wo_fc(x)
        x = torch.flatten(x, 1)
        x = torch.sigmoid(self.fc(x))
        return x

In [11]:
def initialize_model(model, num_classes):
    pass

In [12]:
from torchmetrics.classification import MultilabelJaccardIndex
def test_model(model,testloader):
    correct = 0
    total = 0
    JS = MultilabelJaccardIndex(num_labels = 2)
    with torch.no_grad():
        for data in testloader:
            images, label1,label2 = data
            label1 = torch.from_numpy(np.array([class_names_to_idx[i] for i in label1]))
            label1 = F.one_hot(label1, num_classes=num_classes)
            label1 = label1.to(torch.float32)
            
            label2 = torch.from_numpy(np.array([class_names_to_idx[i] for i in label2]))
            label2 = F.one_hot(label2, num_classes=num_classes)
            label2 = label2.to(torch.float32)
            
            outputs = model(images)
            
            predicted = (outputs.data > 0.5).float()
            correct = 

In [13]:
# Setup 
# BCE loss for multi-label classification
# sigmoid activation after FC layer 
# everything above 0.5 is a predicted label

criterion = nn.BCELoss() 

# get dataset corresponding to each split
train_data = dataset_splits["train"]
intask_val_data = dataset_splits["intask_valid"]
posttask_val_data = dataset_splits["posttask_valid"]
test_data = dataset_splits["test"]

# pre-trained Model on imageNet 
resnet = MultilabelClassifier(n_classes_per_task[0])
seen_classes = 0
# initialize data to train on first task
for task in range(len(tasks)):
    train_data.choose_task(task)
    trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
    seen_classes += n_classes_per_task[task]
    
        
    new_fc = nn.Linear(resnet.num_ftrs, seen_classes)
    
    for cl in range(seen_classes-n_classes_per_task[task]):
        new_fc.weight[cl].data = resnet.fc.weight[cl].data
            
    resnet.fc = new_fc
    resnet = resnet.to(device)
    params_to_update = resnet.parameters()
    if feature_extract:
        params_to_update = []
        for name,param in resnet.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
    else:
        for name,param in resnet.named_parameters():
            if param.requires_grad == True:
                pass

    optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
    
    resnet = train_model(resnet, trainloader, criterion, optimizer_ft , seen_classes,5)

# resnet = train_model(resnet, dataloader_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 1/5
len dataset =  8160
train Loss: 0.2706

Epoch 2/5
len dataset =  8160
train Loss: 0.2212

Epoch 3/5
len dataset =  8160
train Loss: 0.1995

Epoch 4/5
len dataset =  8160
train Loss: 0.1898

Epoch 5/5
len dataset =  8160
train Loss: 0.1800

Training complete in 3m 11s
Epoch 1/5
len dataset =  1840
train Loss: 0.1708

Epoch 2/5
len dataset =  1840
train Loss: 0.0874

Epoch 3/5
len dataset =  1840
train Loss: 0.0802

Epoch 4/5
len dataset =  1840
train Loss: 0.0766

Epoch 5/5
len dataset =  1840
train Loss: 0.0748

Training complete in 0m 42s
Epoch 1/5
len dataset =  2160
train Loss: 0.1482

Epoch 2/5
len dataset =  2160
train Loss: 0.0669

Epoch 3/5
len dataset =  2160
train Loss: 0.0568

Epoch 4/5
len dataset =  2160
train Loss: 0.0543

Epoch 5/5
len dataset =  2160
train Loss: 0.0513

Training complete in 0m 51s
Epoch 1/5
len dataset =  1760
train Loss: 0.1600

Epoch 2/5
len dataset =  1760
train Loss: 0.0604

Epoch 3/5
len dataset =  1760
train Loss: 0.0520

Epoch 4/5
len da